In [21]:
import torch
import torch.nn as nn
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import torch.optim as optim
from ANN import *

In [22]:
df_cleaned=pd.read_csv('Processed_DATA_PITSTRAT.csv')

def preprocess(df_cleaned):

    input_data = df_cleaned[['circuitId', 'raceId', 'points', 'lap','position','laps']].values
    target_data = df_cleaned['stop'].values

    scaler = StandardScaler()
    input_data[:, 2:] = scaler.fit_transform(input_data[:, 2:])

    circuit_encoded = F.one_hot(torch.tensor(input_data[:, 0],dtype=torch.long), num_classes=81)
    race_encoded = F.one_hot(torch.tensor(input_data[:, 1],dtype=torch.long), num_classes=1133)
    points = torch.tensor(input_data[:, 2], dtype=torch.float32).unsqueeze(1)
    lap = torch.tensor(input_data[:, 3], dtype=torch.float32).unsqueeze(1)
    position = torch.tensor(input_data[:, 4], dtype=torch.float32).unsqueeze(1)
    laps = torch.tensor(input_data[:, 5], dtype=torch.float32).unsqueeze(1)


    input_data_encoded = torch.cat([circuit_encoded, race_encoded, points, lap,position,laps], dim=1)
    target_encoded = F.one_hot(torch.tensor(target_data,dtype=torch.long), num_classes=8)
    return input_data_encoded,target_encoded


input_data_encoded,target_encoded=preprocess(df_cleaned)
X_train, X_test, y_train, y_test = train_test_split(input_data_encoded, target_encoded, test_size=0.2, random_state=42)
train_dataset = F1Dataset(X_train, y_train)
test_dataset = F1Dataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
best_loss = float('inf')
circuit_id_max = 81
race_id_max = 1133
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
weight_decay = 1e-5
criterion = nn.CrossEntropyLoss()



cuda


In [24]:
Pitstrat_predictor_model= MLPModel(circuit_id_max=circuit_id_max, race_id_max=race_id_max,dropout_prob=0.2).to(device)
state_dict=(torch.load('../Pitstrat/best_model.pth'))
Pitstrat_predictor_model.load_state_dict(state_dict)

/tmp/ipykernel_380442/2949567155.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict=(torch.load('../Pitstrat/best_model.pth'))


<All keys matched successfully>